In [1]:
import json
import pickle
import numpy as np

In [2]:
with open('ingredients.json', 'r') as fr:
    ingr_list = json.load(fr)
    
with open('clean_data_revised_names.json', 'r') as fr:
    drink_list = json.load(fr)

#print(ingr_list)
#def create_ingr_array

#ingr_nplst = np.array(ingr_list)
#print(ingr_nplst)

#print(drink_list)

In [3]:
#index = np.where(ingr_nplst=='Tang')
#print(index)

In [4]:
ingr_to_num = {}
i=0
for ingr in ingr_list:
    ingr_to_num[ingr] = i
    i=i+1

In [5]:
#print(ingr_dic)

In [6]:
drink_to_num = {}
num_to_drink = {}
j=0
for k,v in drink_list.items():
    drink_to_num[k] = j
    num_to_drink[j] = k
    j=j+1
#print(num_to_drink)

In [7]:
#let's make the drink by ingredients numpy array, so we can eventually pickle it
drink_ingr = np.zeros((len(drink_to_num), len(ingr_to_num)))
for i in range(len(drink_to_num)):
    curr_ingredients = drink_list[num_to_drink[i]]
    #for j in range(len(ingr_to_num)):
    for ingredient in curr_ingredients:
        num_ingredient = ingr_to_num[ingredient[0]]
        drink_ingr[i][num_ingredient] = 1
print(drink_ingr)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
def build_movie_sims_jac(n_mov,input_data):
    """Returns a movie_sims_jac matrix of size (num_movies,num_movies) where for (i,j) :
        [i,j] should be the jaccard similarity between the category sets for movies i and j
        such that movie_sims_jac[i,j] = movie_sims_jac[j,i]. 
        
    Note: Movies sometimes contain *duplicate* categories! You should only count a category once
    
    Params: {n_mov: Integer,
             input_data: List<Dictionary>}
    Returns: Numpy Array 
    """
    # YOUR CODE HERE
    
    final_mat = np.zeros((n_mov,n_mov))
    
    for i in range(n_mov):
        cat1 = set(input_data[i]['categories'])
        for j in range(n_mov):
            cat2 = set(input_data[j]['categories'])
            if i == j:
                final_mat[i][j] = 1
            else:
                intersect = len(set.intersection(cat1, cat2))
                union = len(set.union(cat1, cat2))
                final_mat[i][j] = intersect/union
                
    return final_mat

In [9]:
def drink_to_index(dr):
    return drink_to_num[dr]
def index_to_drink(num):
    return num_to_drink[num]
    
def ingredient_to_index(ingredient_str):
    return ingr_to_num[ingredient_str]
def number_list_of_drinks(ing_str_lst):
    newlst = []
    for ingr in ing_str_lst:
        newlst.append(ingredient_to_index(ingr))     
    return newlst

def get_ingredients_for_drink(drink_num, drink_lst):
    #print(drink_num)
    curr_ingredients = drink_lst[index_to_drink(drink_num)]
    final = []
    for ing in curr_ingredients:
        final.append(ingredient_to_index(ing[0]))
    return final

#returns a numpy array of 1 by size of drink_list. each entry is the jaccard similarity between the user input and drink
def drink_jaccard_sim(user_ingredients_lst, drink_lst):
    final_mat = np.zeros(len(drink_lst))
    #cat1 will be the user set. It should just have a list of numbers that correspond to drinks
    cat1 = set(number_list_of_drinks(user_ingredients_lst))
    for i in range(len(drink_lst)):
        #cat2 will be the ingredients of each individual drink
        cat2 = set(get_ingredients_for_drink(i, drink_lst))
        #do the jaccard
        intersect = len(set.intersection(cat1, cat2))
        union = len(set.union(cat1, cat2))
        final_mat[i] = intersect/union
    return final_mat
    

In [10]:
sample_ingredient_list = ["Banana liqueur",
"Banana rum",
"Banana syrup",
"Batida de Coco",
"Battery",
"Beef bouillon",
"Beer",
"Berries",
"Bitter lemon",
"Bitters",
"Black Sambuca",
"Black pepper",
"Black rum",
"Black vodka",
"Blackberries",
"Blackberry brandy",
"Blackberry schnapps",
"Blackcurrant cordial",
"Blackcurrant schnapps",
"Blackcurrant squash",
"Blackcurrant vodka",
"Blended whiskey",
"Bloody mary mix",
"Blue Curacao",
"Blueberries",
"Blueberry schnapps"]

sim_lst = drink_jaccard_sim(sample_ingredient_list, drink_list)
print(sim_lst)

[0.         0.         0.         ... 0.03571429 0.         0.        ]


In [11]:
def get_top_k_drinks(lst, k):
    #lst should have a list of ranked values, just need to do an argsort for the best ones
    sorted_lst = lst.argsort()[-k:][::-1]
    print(sorted_lst)
    top_drinks = []
    for drink in sorted_lst:
        top_drinks.append(index_to_drink(drink))
    return top_drinks

thing = get_top_k_drinks(sim_lst, 10)
print(thing)

[1196 4964 5528 5500 4016 2888 5029 1478  659 4401]
['Rum Runner #2', 'Blackula', 'Black and Blue', 'Fox River Cocktail', 'Fluxuation Cocktail', 'Smurf fart', 'Blue Bay', 'Fiery Blue Mustang', 'Bazooka Joe', 'Black and Blue #2']


In [13]:
pickle.dump(ingr_list, open( "ingr_lst_pickle.p", "wb" ) )
pickle.dump(drink_ingr, open( "drink_ingr_pickle.p", "wb" ) )
pickle.dump(drink_list, open( "drink_dic_pickle.p", "wb" ) )